In [1]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np


page = urlopen("https://athletics.uwaterloo.ca/sports/football/roster")
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

ul_element = soup.find('ul', class_='sidearm-roster-players')
li_elements = ul_element.find_all('li', class_='sidearm-roster-player')


In [2]:
name_list = []
weight_list = []
position_list = []
height_list = []
elig_year_list = []
program_list = []
acad_year_list = []
hometown_list = []
image_list = []

for player in li_elements:
  player_name = player.find('h3')
  a_tag = player_name.find('a')
  player_name = a_tag.text.strip()
  #print(player_name)
  name_list = name_list + [player_name]

  link = a_tag.get('href')
  link = "https://athletics.uwaterloo.ca/" + link
  #print(link)

  weight = player.find('span', class_='sidearm-roster-player-weight')
  weight = weight.text.strip()
  weight = weight[:-4]
  #print(weight)
  weight_list = weight_list + [weight]

  player_page = urlopen(link)
  player_html = player_page.read().decode("utf-8")
  player_soup = BeautifulSoup(player_html, "html.parser")
  player_div = player_soup.find_all('li')

  position = player_soup.find('div', class_='flex-item-1').find_all('span')[1]
  position = position.text.strip()

  #print(position)
  position_list = position_list + [position]

  height = player_soup.find('div', class_='flex-item-1').find_all('span')[3]
  height = height.text.strip()
  #print(height)
  height_list = height_list + [height]


  elig_year = player_soup.find('div', class_='flex-item-1').find_all('span')[5]
  elig_year = elig_year.text.strip()
  #print(elig_year)
  elig_year_list = elig_year_list + [elig_year]

  program = player_soup.find('div', class_='flex-item-1').find_all('span')[7]
  program = program.text.strip()
  #print(program)
  program_list = program_list + [program]

  acad_year = player_soup.find('div', class_='flex-item-1').find_all('span')[9]
  acad_year = acad_year.text.strip()
  #print(acad_year)
  acad_year_list = acad_year_list + [acad_year]

  hometown = player_soup.find('div', class_='flex-item-1').find_all('span')[11]
  hometown = hometown.text.strip()
  #print(hometown)
  hometown_list = hometown_list + [hometown]

  image = player_soup.find("div", class_="sidearm-roster-player-image")
  if image:
    image = image.find('img')
    image = image.get('src')
    image = "https://athletics.uwaterloo.ca/" + image
    #print(image)
    image_list = image_list + [image]
  else:
    #print("https://i.imgur.com/rxwmSpO.png")
    image_list = image_list + ["https://i.imgur.com/rxwmSpO.png"]
  #print()

In [3]:
name_list = [name.replace('  ', ' ') for name in name_list]

def height_convert(s):
  split_height = s.split("-")
  feet = 30.48 * int(split_height[0])
  inches = 2.54 * int(split_height[1])
  return (feet + inches)
height_list = list(map(height_convert, height_list))

def acad_year_map(y):
  if y[0:3] == "Fir":
    return 1
  elif y[0:3] == "Sec":
    return 2
  elif y[0:3] == "Thi":
    return 3
  elif y[0:3] == "Fou":
    return 4
  elif y[0:3] == "Fif":
    return 5
  else:
    return 6 ## Graduate student
acad_year_list = list(map(acad_year_map, acad_year_list))

In [4]:
print(name_list)
print(weight_list)
print(position_list)
print(height_list)
print(elig_year_list)
print(program_list)
print(acad_year_list)
print(hometown_list)
print(image_list)

['Anthony Miller', 'Anesu Latmore', 'Keesean Bowen', 'Isiah Blackson', 'Nick Orr', 'Ethan Miller', 'Michael Moir', 'Nolan Kaban', 'Nick Morgado', 'Austin Dunnill', 'Yinka Ogunnote', 'Divine Ugoh', 'Isaac White', 'Tyler Tierney', 'John Sargeant', 'Chadley Charles', 'Riley Webster', 'Romell Samuel-Slocombe', 'Hayden Crowe', 'Alex Sooley', 'Owen Herechuk', 'Mason Henry', 'Delroy Njonguo', 'Rachim Blackson', 'Kieran Pama', 'Cole Crossett', 'Logan Heroux', 'Nick Cahyono', 'Isaac Porter', 'Gursiman Dhaliwal', 'Roy Van Der Veen', 'Zach Parmley', 'Isaiah Butts', 'Terry Ojeh', 'Quinten Springer', 'Josh Pileggi', 'James Hinsperger', 'Elijah Pappin', 'Arden Martinez', 'Michael Omoseni', 'Jose Ogunjinmi', 'Jordan Landry', 'Cole Weddell', 'Jadyn Alam-Smith', 'Sam McPherson', 'Nate Mortenson', 'Allen Clements', 'Andrew Kilby', 'Cameron Krug', 'Marcus Miller', 'Ethan Kalra', 'Marcus Palumbo', 'Cameron Bulai', 'Ethan Given', 'Brayden Hastie', 'Devon Martin', 'Matthew Elia', 'Thomas Levings', 'Julian B

In [5]:
import csv
import requests

new_image_list = []

for link in image_list:
  response = requests.get(link, allow_redirects=True)
  new_image_list = new_image_list + [response.url]
  #print(response.url)

image_list = new_image_list
#print(image_list)

In [6]:
import pandas as pd

df = pd.DataFrame({
    'Name': name_list,
    'Position': position_list,
    'Height': height_list,
    'Weight': weight_list,
    'Elig Year': elig_year_list,
    'Acad Year': acad_year_list,
    'Program': program_list,
    'Hometown': hometown_list,
    'Image': image_list

})

#df

In [7]:
df.to_csv('roster.csv', index=False)